In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer as CountV
from sklearn.feature_selection import VarianceThreshold
import scipy
from scipy.sparse import lil_matrix

import numpy as np
import os
# import pandas as pd
import matplotlib.pyplot as plt
import time, random
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.model_selection import train_test_split
import datetime
import argparse
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset
from ember_utils import *
from ember_model import *
from ember_pjr_utils import *



In [ ]:
class Ember_MLP_Net(nn.Module):
    def __init__(self, input_features):
        super(Ember_MLP_Net, self).__init__()
        
        self.fc1 = nn.Linear(input_features, 1024)
        self.fc1_bn = nn.BatchNorm1d(1024)
        self.act1 = nn.ReLU()
        self.fc1_drop = nn.Dropout(p=0.5)
        
        self.fc2 = nn.Linear(1024, 512)
        self.fc2_bn = nn.BatchNorm1d(512)
        self.act2 = nn.ReLU()
        self.fc2_drop = nn.Dropout(p=0.5)
        
        self.fc3 = nn.Linear(512, 256)
        self.fc3_bn = nn.BatchNorm1d(256)
        self.act3 = nn.ReLU()
        self.fc3_drop = nn.Dropout(p=0.5)        
        
        self.fc4 = nn.Linear(256, 128)
        self.fc4_bn = nn.BatchNorm1d(128)
        self.act4 = nn.ReLU()
        self.fc4_drop = nn.Dropout(p=0.5)  
        
        self.fc_last = nn.Linear(128, 1) 
        self.out = nn.Sigmoid()
        
        #self.activate = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        #print(x.shape)
        x = self.fc1(x)
        x = self.fc1_bn(x)
        x = self.act1(x) 
        x = self.fc1_drop(x)

        x = self.fc2(x)
        x = self.fc2_bn(x)
        x = self.act2(x) 
        x = self.fc2_drop(x)
        
        x = self.fc3(x)
        x = self.fc3_bn(x)
        x = self.act3(x) 
        x = self.fc3_drop(x)
        
        x = self.fc4(x)
        x = self.fc4_bn(x)
        x = self.act4(x)
        x = self.fc4_drop(x)
        
        x = self.fc_last(x)
        x = self.out(x)
        return x

def testing_aucscore(model, X_test, Y_test, batch_size, device):
    #X_te = torch.from_numpy(X_test).type(torch.FloatTensor)
    #y_te = torch.from_numpy(Y_test).type(torch.FloatTensor) 
    
    testloader = get_dataloader(X_test, Y_test, batch_size, train_data=False)   
    
    model.eval()
    y_pred_list = []
    y_true_list = []
    test_acc = []
    with torch.no_grad():
        for x_batch, y_batch in tqdm(testloader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            y_test_pred = model(x_batch)
            tmp_test_acc = binary_acc(y_test_pred, y_batch)
            test_acc.append(tmp_test_acc.item())
            
            y_pred_tag = torch.round(y_test_pred).squeeze(1)
            y_pred_list += list(y_pred_tag.cpu().numpy())
            y_true_list += list(y_batch.cpu().numpy())
        
            
    #correct_test_results = (np.array(y_pred_list) == np.array(y_true_list)).sum()
    #acc = correct_test_results/len(y_true_list)
    
    from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score
    
    correct_labels, predicted_labels = np.array(y_true_list), np.array(y_pred_list)
    
    roc_auc = roc_auc_score(correct_labels, predicted_labels)
    precision = precision_score(correct_labels, predicted_labels, average='micro')
    recall = recall_score(correct_labels, predicted_labels, average='micro')
    f1score = f1_score(correct_labels, predicted_labels, average='macro')
    
    print(f'test accuracy {np.mean(test_acc)} and ROC-AUC {roc_auc}')

    #wrong_good, wrong_mal, top_k_mistaken_families = \
    #            get_mistaken_stats(np.array(y_true_list), np.array(np.round(y_pred_list)), Y_test_family, top_k)
    
    return np.mean(test_acc), roc_auc, precision, recall, f1score 

In [ ]:

def get_year_data(data_dir, year, train=True):
    
    if train:
        data_dir = data_dir + '/'
        XY_train = np.load(data_dir + str(year) + '_Domain_AZ_Train_Transformed.npz', allow_pickle=True)
        X_tr, Y_tr = XY_train['X_train'], XY_train['Y_train']
        
        return X_tr, Y_tr
    else:
        data_dir = data_dir + '/'
        XY_test = np.load(data_dir + str(year) + '_Domain_AZ_Test_Transformed.npz', allow_pickle=True)
        X_test, Y_test = XY_test['X_test'], XY_test['Y_test']

        return X_test, Y_test 


def get_MemoryData(X, Y, memory_budget):
    indx = [i for i in range(len(Y))]
    random.shuffle(indx)
    
    replay_index = indx[:memory_budget]
    X_train = X[replay_index]
    Y_train = Y[replay_index]
    
    return X_train, Y_train



def get_GRS_data(data_dir, task_years, memory_budget, train=True, joint=True):
    
    if train:
        X_tr, Y_tr = get_year_data(data_dir, task_years[-1])
        print(f'Current Task Year {task_years[-1]} data X {X_tr.shape} Y {Y_tr.shape}')
        
        if len(task_years) != 1:
            previous_Xs, previous_Ys = [], []
            for year in task_years[:-1]:
                pre_X_tr, pre_Y_tr = get_year_data(data_dir, year)


                pre_X_tr, pre_Y_tr = np.array(pre_X_tr), np.array(pre_Y_tr)
                #print(f'pre_X_tr {pre_X_tr.shape}  pre_Y_tr {pre_Y_tr.shape}')

                for idx, prevSample in enumerate(pre_X_tr):
                    previous_Xs.append(prevSample)
                    previous_Ys.append(pre_Y_tr[idx])


            previous_Xs, previous_Ys = np.array(previous_Xs), np.array(previous_Ys)  

            #print(f'Y_tr {Y_tr.shape}  previous_Ys {previous_Ys.shape}')
            if joint:
                X_tr, Y_tr = np.concatenate((X_tr, previous_Xs)), np.concatenate((Y_tr, previous_Ys))
            else:
                if memory_budget >= len(previous_Ys):
                    X_tr, Y_tr = np.concatenate((X_tr, previous_Xs)), np.concatenate((Y_tr, previous_Ys))
                else:
                    previous_Xs, previous_Ys = get_MemoryData(previous_Xs, previous_Ys, memory_budget)
                    X_tr, Y_tr = np.concatenate((X_tr, previous_Xs)), np.concatenate((Y_tr, previous_Ys))
                    #print(f'memory_budget {memory_budget}  Y_tr {previous_Ys.shape} ')
                    assert memory_budget == len(previous_Ys)
            

        X_train, Y_train  = X_tr, Y_tr
        print(f'X_train {X_train.shape} Y_train {Y_train.shape}\n')
        return X_train, Y_train
    else:
        X_te, Y_te = get_year_data(data_dir, task_years[-1], train=False)
        for year in task_years[:-1]:
            pre_X_te, pre_Y_te = get_year_data(data_dir, year, train=False)
            
            X_te, Y_te = np.concatenate((X_te, pre_X_te)), np.concatenate((Y_te, pre_Y_te))

        X_test, Y_test  = X_te, Y_te
        print(f'X_test {X_test.shape} Y_test {Y_test.shape}')
        return X_test, Y_test



data_dir = '/home/mr6564/continual_research/AZ_Data/Domain_Transformed/' #args.data_dir
num_exps = 1 #args.num_exps
num_epoch = 100 #args.num_epoch
batch_size = 512 #args.batch_size
memory_budget = 'none' #args.memory_budget
patience = 5

exp_seeds = [random.randint(1, 99999) for i in range(1)]

all_task_years = ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']


if args.grs_joint:
    memory_budget = 'joint'

input_features = 1789

replay_type, current_task = 'azdomain', 'azdomain'


cnt =  1    
for exp in exp_seeds:
    start_time = time.time()
    use_cuda = True
    print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
    use_cuda = use_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    torch.manual_seed(exp)

    model = Ember_MLP_Net(input_features)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000001)
       
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    
    model = model.to(device)
    print(f'Model has {count_parameters(model)/1000000}m parameters')    
    criterion = nn.BCELoss()    
    
    
#     standardization = StandardScaler()
#     standard_scaler = None
    for task_year in range(len(all_task_years)):
                
        print(f'\n{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Round {cnt} ...')
        task_start = time.time()
        current_task = all_task_years[task_year]
        task_years = all_task_years[:task_year+1]
        print(f'Current Task {current_task} with Budget {memory_budget}')


        model_save_dir = '../az_model/GRS_SavedModel' + '/GRSModel_' + str(memory_budget) + '/' + str(current_task) + '/'
        create_parent_folder(model_save_dir)
        
        opt_save_path = '../az_model/GRS_SavedModel' + '/GRSOpt_' + str(memory_budget) + '/' + str(current_task) + '/'
        create_parent_folder(opt_save_path)
        
        results_save_dir =  '../az_model/GRS_SavedResults_' +'/GRS_' + str(memory_budget) + '/' 
        create_parent_folder(results_save_dir)
        
        
        if args.grs_joint:
            X_train, Y_train = get_GRS_data(data_dir, task_years, memory_budget, train=True, joint=True)
            X_test, Y_test = get_GRS_data(data_dir, task_years, memory_budget, train=False, joint=True)
        else:
            X_train, Y_train = get_GRS_data(data_dir, task_years, memory_budget, train=True, joint=False)
            X_test, Y_test = get_GRS_data(data_dir, task_years, memory_budget, train=False, joint=False)
        
        
        print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Standardizing ...')
        
#         if args.grs_joint:
#                 standardization = StandardScaler()
#                 standard_scaler = None
#                 standard_scaler = standardization.fit(X_train)
#         else:        
#                 standard_scaler = standardization.partial_fit(X_train)

#         X_train = standard_scaler.transform(X_train)
#         X_test = standard_scaler.transform(X_test)


        X_train, Y_train = np.array(X_train, np.float32), np.array(Y_train, np.int32)
        X_test, Y_test = np.array(X_test, np.float32), np.array(Y_test, np.int32)        
        
        print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Training ...')
        
        
        
        task_training_time, epoch_ran, training_loss, validation_loss  = training_early_stopping(\
                                     model, model_save_dir, opt_save_path, X_train, Y_train,\
                                     X_test, Y_test, patience, batch_size, device, optimizer, num_epoch,\
                                     criterion, replay_type, current_task, exp, earlystopping=True)


        end_time = time.time()
        print(f'Elapsed time {(end_time - start_time)/60} mins.') 


        best_model_path = model_save_dir + os.listdir(model_save_dir)[0]
        print(f'loading best model {best_model_path}')
        model.load_state_dict(torch.load(best_model_path))

        #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.000001)
        best_optimizer = opt_save_path + os.listdir(opt_save_path)[0]
        print(f'loading best optimizer {best_optimizer}')
        optimizer.load_state_dict(torch.load(best_optimizer))


        acc, rocauc, precision, recall, f1score = testing_aucscore(model, X_test, Y_test, batch_size, device)
        end_time = time.time()
        print(f'Elapsed time {(end_time - start_time)/60} mins.')    
        
       
        results_f = open(os.path.join('./Submission_Domain/' + 'grs_' + str(memory_budget) + '_results.txt'), 'a')
        result_string = '{}\t{}\t{}\t{}\t{}\t\n'.format(current_task, acc, precision, recall, f1score)
        
        results_f.write(result_string)
        results_f.flush()
        results_f.close()
    
    end_time = time.time()
    cnt += 1
    print(f'Elapsed time {(end_time - start_time)/60} mins.')
    
    del model_save_dir
    del opt_save_path
    del results_save_dir
